# Ordinary Least Squares

$$y_i = \beta_0 + x_i \beta + \epsilon_i$$

+ $y$ is $n \times 1$; values of the dependent variable are observed
+ $\beta_0$ is also $n \times 1$; all values are 1
+ $\beta$ is $k \times 1$; we are estimating this parameter
+ $x$ is $n \times k$; values of the independent variables are observed
+ $\epsilon$ is $n \times 1$; assumed to be normally distributed centered on 0

In [58]:
from matplotlib.figure import Figure
from matplotlib.axes import Axes
import seaborn as sb
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_probability as tfp
import statsconcepts
from patsy import dmatrices

%load_ext autoreload
%aimport statsconcepts.ols, statsconcepts.viz
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The OLS estimator is derived from assertions we make about the estimator we want to see. For example, suppose we think it's wise to minimize the error associated with an estimate. We must choose a *loss function* that is consistent with that goal. We could choose to minimize the absolute sum of errors:

$$L_{\text{abs}} = \text{min} \Sigma_i^n e_i$$

However, this leaves us exposed large errors that offset each other because they differ in sign. By squaring the errors, we assert that we care about overall error magnitude. As a (potentially useful) byproduct, larger errors are penalized more than smaller errors. The OLS estimator is predicated on these design objectives, which are encoded in the loss function.

$$L_{\text{OLS}} = \text{min} \Sigma_i^n e_i^2$$

We can derive our estimator $\beta$ by isolating $\epsilon$ and squaring it to mimic the loss function. Then we need only differentiate with respect to $\beta$ to find value of beta that is associated with the minimum error. Said differently, if we allow $\beta$ to vary, what value yields the smallest gap between our observed dependent values $y$ and our estimated values $X'\beta$?

$$
\begin{align}
y_i &= x_i \beta  + \epsilon_i \\
e_i &= y_i - x_i \beta \\
e_i^2 &= y_i^2 - 2y_i x_i \beta + x_i^2 \beta^2 \\
\frac{\partial}{\partial \beta} e_i^2 &= \frac{\partial}{\partial \beta}y_i^2 - 2y_i x_i \beta + x_i^2 \beta^2 \\
0 &= -2y_i x_i + 2x_i^2 \beta \\
2x_i^2 \beta &= 2y_i x_i \\ 
\beta &= \frac{y_i x_i}{x_i^2} \\
\end{align}
$$

We have solved the scalar valued case, and we can do so at the matrix level as well.

$$
\begin{align}
    y &= X' \beta  + \epsilon \\
    \epsilon &= y - X' \beta \\
    \epsilon'\epsilon &= y'y - 2X'y \beta + \beta'X'X \beta \\
    \frac{\partial}{\partial \beta} \epsilon'\epsilon &= \frac{\partial}{\partial \beta}y'y - 2X'y \beta + \beta'X'X \beta \\
    0 &= -2X'y + 2X'X \beta \\
    2X'X \beta &= 2X'y \\ 
    \beta &= (X'X)^{-1}X'y
\end{align}
$$

Alternatively, we can derive the OLS estimator by way of the orthogonality condition which asserts that the error term must be uncorrelated with the regressors: $E[X'e] = 0$.

$$
\begin{align}
    y &= X'\beta + \epsilon \\
    \epsilon &= y - X'\beta \\
    X'\epsilon &= X'(y - X'\beta) \\
    X'\epsilon &= X'y - X'X'\beta \\
    0 &= X'y - X'X'\beta \\
    X'X'\beta &= X'y  \\
    \beta &= (X'X)^{-1}X'y
\end{align}
$$

In [51]:
def algebraically_closed_data(b0: tf.Tensor, b: tf.Tensor, x: tf.Tensor) -> xr.Dataset:
    nrows: int
    ncols: int
    nrows, ncols = x.shape
    
    b0_tensor: tf.Tensor = tf.fill([nrows, 1], b0)
    b_reshaped: tf.Tensor = tf.reshape(b, shape=(nrows, 1))
    y: tf.Tensor = tf.reshape(b0_tensor + tf.matmul(x, b_reshaped), shape=(nrows,))
    
    y_arr: xr.DataArray = xr.DataArray(
        data=y,
        coords={"obs": range(nrows)},
        dims=["obs"]
    )
    x_arr: xr.DataArray = xr.DataArray(
        data=x,
        coords={"obs": range(nrows), "cols": range(ncols)},
        dims=["obs", "cols"]
    )
    out: xr.Dataset = xr.Dataset({
        "y": y_arr,
        "x": x_arr
    })
    return out
    
test: xr. Dataset = algebraically_closed_data(tf.constant(10), tf.constant([1, 2]), tf.constant([3,4,5,6], shape=(2,2)))

[autoreload of statsconcepts.ols failed: Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/tf/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/Caskroom/miniconda/base/envs/tf/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/Caskroom/miniconda/base/envs/tf/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/Caskroom/miniconda/base/envs/tf/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importli

## Input Data

In [4]:
df: pd.DataFrame = sb.load_dataset("tips")
df.index.name = "transaction"
data: xr.Dataset = xr.Dataset(df)
    
data

<xarray.Dataset>
Dimensions:      (transaction: 244)
Coordinates:
  * transaction  (transaction) int64 0 1 2 3 4 5 6 ... 238 239 240 241 242 243
Data variables:
    total_bill   (transaction) float64 16.99 10.34 21.01 ... 22.67 17.82 18.78
    tip          (transaction) float64 1.01 1.66 3.5 3.31 ... 2.0 2.0 1.75 3.0
    sex          (transaction) object 'Female' 'Male' 'Male' ... 'Male' 'Female'
    smoker       (transaction) object 'No' 'No' 'No' 'No' ... 'Yes' 'No' 'No'
    day          (transaction) object 'Sun' 'Sun' 'Sun' ... 'Sat' 'Sat' 'Thur'
    time         (transaction) object 'Dinner' 'Dinner' ... 'Dinner' 'Dinner'
    size         (transaction) int64 2 3 3 2 4 4 2 4 2 2 ... 2 2 2 2 3 3 2 2 2 2